# Codeasy Task - YOLOv8

To start with, install the [Ultralytics](https://github.com/ultralytics/ultralytics) and [Fiftyone](https://docs.voxel51.com/getting_started/install.html).

In [1]:
!pip install fiftyone ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.8/750.8 kB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.9/676.9 kB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

After succesfully installing the required packages, import the necessary libraries.

In [2]:
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F
import numpy as np
import os

Migrating database to v0.23.8


INFO:fiftyone.migrations.runner:Migrating database to v0.23.8


From ultralytics, YOLOv8 model will be imported, downloaded, in this case that is yolov8n model.

In [3]:
from ultralytics import YOLO

detection_model = YOLO("yolov8n.pt")

100%|██████████| 6.23M/6.23M [00:00<00:00, 106MB/s]


As the data that will be used is not in format of YOLOv8 (YOLOv5) requirements, custom function that uses Fiftyone export function will be use to export it in the desired format.

In [4]:
# custom function which exports the data in yolo format
def export_yolo_data(
    samples,
    export_dir,
    classes,
    label_field = "ground_truth",
    split = None
    ):

    if type(split) == list:
        splits = split
        for split in splits:
            export_yolo_data(
                samples,
                export_dir,
                classes,
                label_field,
                split
            )
    else:
        if split is None:
            split_view = samples
            split = "val"
        else:
            split_view = samples.match_tags(split)

        split_view.export(
            export_dir=export_dir,
            dataset_type=fo.types.YOLOv5Dataset,
            label_field=label_field,
            classes=classes,
            split=split
        )

To access the data that will be used in this project, library Fiftyone.zoo will be used and its load_zoo_dataset function which has the Open Images v7 dataset. While the dataset contains millions of pictures, for the training of the model that will be used in this project, only 2 classes will be needed, that being Owl and Sheep. To only get the pictures that contain the desired classes, in the classes variable we insert the desired classes and after map the dataset adding the "ground_truth" and giving the classes of "Owl" and "Sheep" a new class name of "owl" and "sheep". Name of the class can be change to anything user wants as the name.

In [5]:
# creating the custom dataset
custom_dataset = foz.load_zoo_dataset(
    "open-images-v7",
    classes = ["Owl", "Sheep"],
    only_matching=True,
    label_types="detections"
).map_labels(
    "ground_truth",
    {"Owl": "owl",
     "Sheep": "sheep"
     }
)

INFO:fiftyone.zoo.datasets:Downloading split 'train' to '/root/fiftyone/open-images-v7/train' if necessary


INFO:fiftyone.utils.openimages:Downloading 'https://storage.googleapis.com/openimages/2018_04/train/train-images-boxable-with-rotation.csv' to '/root/fiftyone/open-images-v7/train/metadata/image_ids.csv'


 100% |██████|    4.8Gb/4.8Gb [5.8s elapsed, 0s remaining, 619.7Mb/s]       


INFO:eta.core.utils: 100% |██████|    4.8Gb/4.8Gb [5.8s elapsed, 0s remaining, 619.7Mb/s]       


INFO:fiftyone.utils.openimages:Downloading 'https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv' to '/root/fiftyone/open-images-v7/train/metadata/classes.csv'


...

INFO:fiftyone.zoo.datasets:Loading 'open-images-v7' split 'validation'


 100% |███████████████████| 92/92 [283.8ms elapsed, 0s remaining, 324.1 samples/s]     


INFO:eta.core.utils: 100% |███████████████████| 92/92 [283.8ms elapsed, 0s remaining, 324.1 samples/s]     


Dataset 'open-images-v7' created


INFO:fiftyone.zoo.datasets:Dataset 'open-images-v7' created


So the dataset doesn't get deleted, using persisent = True on the dataset will keep it even if the database is shut down.

In [6]:
custom_dataset.persistent = True

After making the dataset that contains the relevant pictures, next task is to split the dataset to train set and validation set, in this case in ratio of 0.8 and 0.2. First the tags are deleted and after that random split is called so the data is randomized in each set giving the model better representaion of data. After that, already mentioned function which exports the sets in YOLO format is called and custom_train dataset is made with only 2 specified classes and train/val split.

In [8]:
import fiftyone.utils.random as fornd

classes = ["owl", "sheep"]

## delete existing tags to start fresh
custom_dataset.untag_samples(custom_dataset.distinct("tags"))

## split into train and val
fornd.random_split(
    custom_dataset,
    {"train": 0.8, "val": 0.2}
)

## export in YOLO format
export_yolo_data(
    custom_dataset,
    "custom_train",
    classes,
    split = ["train", "val"]
)

 100% |███████████████| 2327/2327 [16.9s elapsed, 0s remaining, 188.6 samples/s]      


INFO:eta.core.utils: 100% |███████████████| 2327/2327 [16.9s elapsed, 0s remaining, 188.6 samples/s]      


Directory 'custom_train' already exists; export will be merged with existing files


 100% |█████████████████| 582/582 [2.8s elapsed, 0s remaining, 177.1 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 582/582 [2.8s elapsed, 0s remaining, 177.1 samples/s]      


After the dataset to train model is ready, using the !yolo command and attributes such as:
*   task - what model needs to do
*   mode - in which mode the model is set
*   model - which model is used to train
*   data - specifies the path to the dataset configuration file
*   epochs - number of training epochs, in this example dataset will be passed through the model 2 times during training.
*   imgsz - sets the input image size for training, in this exampled it is set to 640x640 pixels
*   batch - specifies how many samples, images will be processed in each iteration or batch during training, in this example it is set to 16 which means that the model will process 16 images in parallel before updating its weights based on the computed gradients.







In [9]:
!yolo task=detect mode=train model=yolov8n.pt data=custom_train/dataset.yaml epochs=2 imgsz=640 batch=16

Ultralytics YOLOv8.2.2 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=custom_train/dataset.yaml, epochs=2, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=

After training is complete, model that performed the best on validation set is saved as the best.pt and is ready to be used or maybe trained again to get better results.

In [10]:
# saving the best.pt model in variable model
model = YOLO('/content/runs/detect/train/weights/best.pt')

As the Google Colab deletes the files after session, exporting the model and saving it on your local computer or cloud server will allow for further uses even after this session is over. In this project model will be exported in default settings, further information on exporting can be found [here](https://docs.ultralytics.com/modes/export/#export-formats).

In [ ]:
# exporting model
model.export()

Model can be tested on a simple picture using the !yolo command and atributes such as:


*   model - model used to predict
*   source - specifies the path to the image/video source

In [14]:
!yolo predict model="/content/runs/detect/train/weights/best.pt" source='/content/test2.jpeg'

Ultralytics YOLOv8.2.2 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs

image 1/1 /content/test2.jpeg: 608x640 1 owl, 271.3ms
Speed: 23.6ms preprocess, 271.3ms inference, 2136.9ms postprocess per image at shape (1, 3, 608, 640)
Results saved to runs/detect/predict2
💡 Learn more at https://docs.ultralytics.com/modes/predict


Test result can be accessed in the runs/detect/predict folder.